# Manual Analysis of AgentIssue-Bench: Why Do Agents Fail?

This notebook provides a human-perspective analysis of agent failures on real-world agent system bugs, examining logs from Auto-Code-Rover to understand failure patterns.

**Analysis Framework:**
1. Bug Reproduction
2. Bug Localization
3. Fix Generation
4. Fix Validation


## 1. Setup and Data Loading


In [ ]:
import json
import pandas as pd
from pathlib import Path
import re

# Load issue data
with open('auto-code-rover/agent_issue.json', 'r') as f:
    issues = json.load(f)

print(f"Total issues in dataset: {len(issues)}")
print(f"\nFirst issue example:")
print(f"ID: {issues[0]['instance_id']}")
print(f"Title: {issues[0]['title']}")


## 2. Issue Categorization by Type

Let's categorize issues by their root cause to understand the dataset composition.


In [ ]:
# Categorize issues
categories = {
    'dependency': [],
    'configuration': [],
    'logic_error': [],
    'type_handling': [],
    'state_management': []
}

# Simple keyword-based categorization
for issue in issues[:20]:  # Analyze first 20
    title = issue['title'].lower()
    problem = issue['problem_statement'].lower()
    
    if 'modulenotfounderror' in problem or 'no module named' in problem:
        categories['dependency'].append(issue['instance_id'])
    elif 'config' in problem or 'yaml' in problem or 'json' in problem:
        categories['configuration'].append(issue['instance_id'])
    elif 'keyerror' in problem or 'attributeerror' in problem:
        categories['logic_error'].append(issue['instance_id'])
    elif 'type' in problem or 'boolean' in problem:
        categories['type_handling'].append(issue['instance_id'])
    else:
        categories['state_management'].append(issue['instance_id'])

# Display results
for cat, items in categories.items():
    print(f"{cat}: {len(items)} issues")
    if items:
        print(f"  Examples: {items[:3]}")


## 3. Deep Dive: Case Study - Dependency Issue (lagent-279)

**Issue:** Missing `tenacity` dependency

**From human perspective, this should be EASY to fix:**
- Clear error message pointing to missing module
- Solution: Add `tenacity` to requirements.txt
- 1-line fix


In [ ]:
# Examine lagent-279 issue
lagent_issue = [i for i in issues if 'lagent-279' in i['instance_id']][0]

print("="*60)
print("CASE STUDY: lagent__lagent-279")
print("="*60)
print(f"\nTitle: {lagent_issue['title']}")
print(f"\nError (excerpt):")
print(lagent_issue['problem_statement'][:300])

print("\n" + "="*60)
print("HUMAN ANALYSIS:")
print("="*60)
print("✅ Clear reproduction: Just run the command")
print("✅ Easy localization: Error points directly to missing module")  
print("✅ Simple fix: Add 'tenacity' to requirements/runtime.txt")
print("✅ Easy validation: Run command again")
print("\n❓ Why agents might struggle:")
print("   - Need to understand project dependency management")
print("   - Must know which requirements file to modify")


## 4. Agent Failure Analysis: Patch Outcomes

Now let's examine actual agent outcomes to see success rates.


In [ ]:
# Count different patch outcomes
patch_outcomes = {
    'applicable_patch': 0,
    'raw_patch_but_unmatched': 0,
    'raw_patch_but_unparsed': 0,
    'matched_but_empty': 0
}

results_dir = Path('auto-code-rover/results/acr-run-1')
for outcome_dir in results_dir.iterdir():
    if outcome_dir.is_dir() and outcome_dir.name in patch_outcomes:
        count = len(list(outcome_dir.iterdir()))
        patch_outcomes[outcome_dir.name] = count

print("Agent Patch Generation Outcomes:")
print("="*60)
for outcome, count in patch_outcomes.items():
    print(f"{outcome:30s}: {count:3d} issues")

total = sum(patch_outcomes.values())
success_rate = (patch_outcomes['applicable_patch'] / total * 100) if total > 0 else 0
print(f"\nSuccess rate: {success_rate:.1f}%")
print(f"\n💡 Insight: Only ~{success_rate:.0f}% of patches are directly applicable!")
